In [1]:
%run ../../utils/commonImports.py
%run ../../utils/tradingImports.py
%matplotlib inline

from base import supres, BaseStrategy
from exputils import *

# Load Data

In [2]:
# dropbox_dir = 'D:\\Dropbox\\My work\\krypl-project'
dropbox_dir = '/Users/david.vesely/Dropbox/My work/krypl-project'

In [3]:
pairs = pd.read_csv('selected-pairs.tsv', header=None)[0].tolist()
train_dict = {pair: read_train(pair, dropbox_dir) for pair in pairs}

# Experiment setup

In [4]:
from itertools import product


features = ['support']
window_sizes = [4, 6, 8, 12, 24, 30, 48]

stop_losses = list(np.linspace(0, 0.1, 11))[1:]
returns = [i / 100 for i in range(1, 6)]

args = product(pairs, window_sizes, stop_losses, returns)
args = list(args)
len(args)

6650

# Try params

In [5]:
supports_for_data_dict = load_model('ohlc_supports_train.pkl')

In [6]:
def trading_stats(data, pair, willing_loss, _return):
    contract_pair = ContractPair.new(*[x.lower() for x in pair.split('_')])
    dm = OhlcDataManager(data[['open', 'low', 'high', 'close']], data[features])
    wallet = {'btc': 100}
    exchange = BackTestExchange(dm, wallet, 0.0025)
    strategy = BaseStrategy(exchange, 
                            dm, 
                            contract_pair, 
                            willing_loss=willing_loss,  
                            target_return=_return)
    strategy.trade()
    strategy_stats = strategy.stats(contract_pair['priceContract']).report()
    return strategy_stats


def trading_stats_f(args):
    pair, window_size, stop_loss, _return = args
    
    data = train_dict[pair]
    data['support'] = supports_for_data_dict[pair][window_size]
    stats = trading_stats(data, pair, stop_loss, _return)
    return pair, window_size, stop_loss, _return, stats

In [10]:
results = run_parallel(trading_stats_f, args)

param_cols = ['pair', 'window_size', 'stop_loss', '_return']
stats_cols = ['startAmount', 'numberOfTrades', 'totalProfit', 'avgProfit',
    'winPercentage', 'avgWinTrade', 'avgLossTrade', 'profitFactor',
    'maxDrawdown', 'avgTimeToClose', 'minTimeToClose', 'maxTimeToClose']

result_df = pd.DataFrame(results, columns=param_cols+['stats'])

for c in stats_cols:
    result_df[c] = result_df['stats'].apply(lambda x: x.iloc[:, 0][c])
    
result_df = result_df.drop('stats', axis=1)

rpath = os.path.join(dropbox_dir,'result', 'base-strategy3-learn-01.pkl')
result_df.to_pickle(rpath)

# Tmp

In [49]:
from modeling.strategy import Strategy

In [47]:
pair = 'BTC_ETH'
data = train_dict[pair]
data['support'] = supports_for_data_dict[pair][12]
willing_loss = .05
_return = -0.02

trading_stats(data, pair, willing_loss, _return)

1 [0.02, 0.03, 0.05] 0.04691601562936343 -0.02
1 [0.03, 0.04, 0.06] 0.0566707782792959 -0.03
1 [0.03, 0.04, 0.05] 0.053534172695458015 -0.03
1 [0.03, 0.04, 0.06] 0.055486260238918894 -0.03
0 [0.08, 0.11, 0.16] 0.15919283585713584 0.05
1 [0.02, 0.03] 0.03123982070166076 -0.02
1 [0.06, 0.08, 0.12] 0.12109073295893391 -0.06
1 [0.01, 0.02, 0.03] 0.025221023335383008 -0.01
1 [0.02, 0.03] 0.034620242349497454 -0.02
1 [0.01, 0.02] 0.023003074900828535 -0.01
1 [0.1, 0.14, 0.2] 0.20045321446650832 -0.1
1 [0.11, 0.15, 0.22] 0.21642193686386435 -0.11
1 [0.04, 0.05, 0.07] 0.07151896036875649 -0.04
1 [0.03, 0.04, 0.06] 0.05802586387081998 -0.03
1 [0.04, 0.05, 0.07] 0.07148861968818787 -0.04
1 [0.05, 0.07, 0.11] 0.10687735919725616 -0.05
1 [0.02, 0.03] 0.0332943648583125 -0.02
1 [0.04, 0.06, 0.09] 0.08623567312059255 -0.04
1 [0.03, 0.05, 0.07] 0.06664586097730951 -0.03
1 [0.03, 0.04, 0.05] 0.054788958259028075 -0.03
0 [0.14, 0.19, 0.27] 0.2709848422683273 0.05
1 [0.03, 0.04, 0.05] 0.0519840035184504

,btc
startAmount,100.0000
numberOfTrades,213.0000
totalProfit,490.9472
avgProfit,2.3049
winPercentage,95.7746
avgWinTrade,2.6056
avgLossTrade,-4.5107
profitFactor,13.0934
maxDrawdown,4.1034
avgTimeToClose,2.1573
